# Implémentation de méthodes élémentaires pour la classification supervisée : Naive Bayes et classifieur par plus proches voisins

Pour ce TP, nous aurons besoin des modules Python ci-dessous, il vous faut donc évidemment exécuter cette première cellule.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
from sklearn.metrics import confusion_matrix

Le jeu de données [Vertebral Column](https://archive.ics.uci.edu/ml/datasets/Vertebral+Column) permet d'étudier les pathologies d'hernie discale et de Spondylolisthesis. Ces deux pathologies sont regroupées dans le jeu de données en une seule catégorie dite `Abnormale`. 

Il s'agit donc d'un problème de classification supervisée à deux classes :
- Normale (NO) 
- Abnormale (AB)    

avec 6 variables bio-mécaniques disponibles (features).

L'objectif du TP est d'implémenter quelques méthodes simples de classification supervisée pour ce problème.

# Importation des données

> Télécharger le fichier column_2C.dat depuis le site de l'UCI à [cette adresse](https://archive.ics.uci.edu/ml/datasets/Vertebral+Column). 
>
> On peut importer les données sous python par exemple avec la librairie [pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html). Vous pourrez au besoin consulter la documentation de la fonction [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html). 
> 
> Le chemin donné dans la fonction `read_csv`est une chaîne de caractère qui spécifie le chemin complet vers le ficher sur votre machine. On peut aussi donner une adresse url si le fichier est disponible en ligne.
>
> Attention à la syntaxe pour les chemins sous Windows doit etre de la forme  `C:/truc/machin.csv`. 
> 
> Voir ce [blog](https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f) pour en savoir plus sur la "manipulation des chemins" sur des OS variés. 

In [2]:
file_path= "E:/Centrale Nantes/AI/A2/STASC/TP/TP3/vertebral+column/column_2C.dat"
Vertebral = pd.read_csv(file_path,
                          delim_whitespace = True,
                          header = 20)

Vertebral.columns = ["pelvic_incidence", 
                     "pelvic_tilt", 
                     "lumbar_lordosis_angle", 
                     "sacral_slope","pelvic_radius",
                     "degree_spondylolisthesis",
                     "class"]


> Vérifier à l'aide des méthodes `.head()`  et `describe()` que les données sont bien importées.

In [3]:
Vertebral.head()

summary = Vertebral.describe()
print(summary)

       pelvic_incidence  pelvic_tilt  lumbar_lordosis_angle  sacral_slope  \
count        289.000000   289.000000             289.000000    289.000000   
mean          61.383875    17.712145              53.099827     43.672145   
std           17.271718    10.225877              18.523934     13.478980   
min           26.150000    -6.550000              14.000000     13.370000   
25%           47.740000    10.540000              39.000000     33.930000   
50%           60.040000    16.480000              51.000000     43.000000   
75%           74.430000    22.230000              64.000000     53.130000   
max          129.830000    49.430000             125.740000    121.430000   

       pelvic_radius  degree_spondylolisthesis  
count     289.000000                289.000000  
mean      118.108616                 28.000069  
std        13.428337                 38.317024  
min        70.080000                -11.060000  
25%       110.710000                  1.740000  
50%       11

> Les librairies de Machine Learning telles que `sckitlearn` prennent en entrée des tableau numpy (pas des objets pandas). Créer un tableau numpy que vous nommerez `VertebralVar` pour les features et un vecteur numpy `VertebralClas` pour la variable de classe. Voir par exemple [ici](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_numpy.html#pandas.DataFrame.to_numpy).

In [4]:
VertebralVar  = Vertebral[["pelvic_incidence", 
                     "pelvic_tilt", 
                     "lumbar_lordosis_angle", 
                     "sacral_slope","pelvic_radius",
                     "degree_spondylolisthesis"]].to_numpy()

VertebralClas = Vertebral["class"].to_numpy()

# Découpage train / test

En apprentissage statistique, classiquement un prédicteur est ajusté sur une partie seulement des données et l'erreur de ce dernier est ensuite évaluée sur une autre partie des données disponibles. Ceci permet de ne pas utiliser les mêmes données pour ajuster et évaluer la qualité d'un prédicteur. Cette problématique est l'objet du prochain chapitre.

> En utilisant la fonction [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) de la librairie [`sklearn.model_selection`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection), sélectionner aléatoirement 60% des observations pour l'échantillon d'apprentissage et garder le reste pour l'échantillon de test. 

In [5]:
from sklearn.model_selection import train_test_split
VertebralVar_train,VertebralVar_test,VertebralClas_train, VertebralClas_test = train_test_split(VertebralVar, VertebralClas, test_size=0.4, random_state=42)
ntot = len(VertebralVar)     # Calcule la longueur totale de l'échantillon complet (ntot)
ntrain = len(VertebralVar_train)     # Calcule la longueur de l'échantillon d'apprentissage (ntrain)
ntest = len(VertebralVar_test)### longueur totale de l'échantillon de test -TO DO ####

Remarque : on peut aussi le faire à la main avec la fonction [`sklearn.utils.shuffle`](https://scikit-learn.org/stable/modules/generated/sklearn.utils.shuffle.html).

# Extraction des deux classes

> Extraire les deux sous-échantillons de classes respectives "Abnormale" et "Normale" pour les données d'apprentissage et de test.

In [6]:
ab_index = (VertebralClas_train == 'AB')
no_index = (VertebralClas_train == 'NO')
VertebralVar_train_AB = VertebralVar_train[ab_index]
VertebralVar_train_NO = VertebralVar_train[no_index]
print(VertebralClas_train)

['AB' 'AB' 'AB' 'NO' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'NO' 'NO'
 'AB' 'AB' 'AB' 'NO' 'NO' 'AB' 'AB' 'AB' 'NO' 'AB' 'NO' 'AB' 'AB' 'AB'
 'NO' 'AB' 'AB' 'AB' 'AB' 'NO' 'AB' 'AB' 'AB' 'NO' 'NO' 'AB' 'NO' 'NO'
 'AB' 'AB' 'NO' 'AB' 'AB' 'AB' 'NO' 'AB' 'NO' 'NO' 'AB' 'AB' 'AB' 'AB'
 'NO' 'AB' 'NO' 'AB' 'NO' 'NO' 'AB' 'NO' 'AB' 'AB' 'AB' 'AB' 'AB' 'NO'
 'NO' 'AB' 'AB' 'AB' 'AB' 'NO' 'NO' 'AB' 'NO' 'AB' 'AB' 'NO' 'AB' 'AB'
 'AB' 'NO' 'AB' 'AB' 'NO' 'NO' 'AB' 'NO' 'AB' 'NO' 'NO' 'NO' 'AB' 'AB'
 'AB' 'NO' 'NO' 'NO' 'AB' 'AB' 'NO' 'AB' 'NO' 'NO' 'NO' 'AB' 'AB' 'AB'
 'AB' 'AB' 'NO' 'AB' 'AB' 'AB' 'AB' 'NO' 'AB' 'NO' 'AB' 'AB' 'NO' 'NO'
 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'NO' 'NO' 'AB' 'AB' 'AB' 'AB' 'AB'
 'AB' 'NO' 'AB' 'NO' 'NO' 'AB' 'AB' 'AB' 'NO' 'AB' 'AB' 'AB' 'AB' 'NO'
 'NO' 'AB' 'AB' 'NO' 'NO' 'AB' 'AB' 'AB' 'AB' 'AB' 'NO' 'AB' 'NO' 'AB'
 'AB' 'AB' 'AB' 'NO' 'AB']


In [7]:
n_AB = len(VertebralVar_train_AB)
n_NO = len(VertebralVar_train_NO)
print(n_AB)
print(n_NO)

113
60


# Gaussian Naive Bayes

Nous allons ajuster un classifieur naif bayesien sur les données d'apprentissage.

Pour une observation $x \in \mathbb R^6$, la régle du MAP consiste à choisir la catégorie $\hat y (x) = \hat k $ qui maximise (en $k$) 
$$ score_k(x) = \hat \pi_k \prod_{j=1} ^6  \hat f_{k,j}(x_j)   $$
où :
- $k$ est le numéro de la classe ;
- $\hat \pi_k$ est la proportion observée de la classe $k$, 
- $\hat f_{k,j} $ est la densité gaussienne univariée de la classe $k$ pour la variable $j$. Les paramètres de cette loi valent (ajustés par maximum de vraisemblance) :
    - $\hat \mu_{k,j}$ : la moyenne empirique de la variable $X^j$ restreinte à la classe k,
    - $ \hat \sigma^2_{k,j}$ : la variance empirique de la variable $X^j$ restreinte à la classe k.
    
Noter que la fonction $x \mapsto  \prod_{j=1} ^6  f_{k,j}(x_j) $ peut aussi être vue comme une densité gaussienne multidimensionnelle de moyenne $(\mu_{k,1}, \dots, \mu_{k,6})$ et de matrice de covariance diagonale $diag(\hat \sigma^2_{k,1},\dots,\hat  \sigma^2_{k,6})$. Cette remarque évite de devoir calculer le produit de 6 densités univariées, à la place on calcule plus directement la valeur de la densité multidimensionnelle.

Pour calculer la valeur de la densité d'une gaussienne multidimensionnelle en un point $x$ de $\mathbb R ^d$ on peut utililser la fonction [`multivariate_normal`](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.multivariate_normal.html) de la librairie [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html). 

On pourra utiliser la fonction `var` de numpy pour calculer le vecteur des variances.

Calcul des moyennes et des variances de chaque variable pour chacun des deux groupes :

In [8]:
mean_AB = np.mean(VertebralVar_train_AB, axis = 0)
mean_NO = np.mean(VertebralVar_train_NO, axis = 0)

# variances estimées variable par variable pour AB (sur le train) :
var_AB = np.var(VertebralVar_train_AB, axis = 0)
# variances estimées variable par variable pour NO (sur le train) :
var_NO = np.var(VertebralVar_train_NO, axis = 0)

# on forme les matrices de covariance (matrices diagonales car indep) :
Cov_NB_AB =  np.diag(var_AB)
Cov_NB_NO =  np.diag(var_NO)


Calcul du "score" sur chaque groupe pour chaque element des données test : 

In [9]:
from sklearn.naive_bayes import GaussianNB
from scipy.stats import multivariate_normal
import numpy as np

# Créer un modèle Bayesien naïf gaussien
gnb = GaussianNB()

# Utiliser les données d'entraînement pour ajuster le modèle
gnb.fit(VertebralVar_train, VertebralClas_train)

# Utiliser le modèle pour faire des prédictions de classe sur les données de test
pred_NB_test = gnb.predict(VertebralVar_test)

# Obtenez le nombre d'échantillons des catégories "AB" et "NO" dans les données d'entraînement
num_samples_AB = np.sum(VertebralClas_train == 'AB')
num_samples_NO = np.sum(VertebralClas_train == 'NO')


# Calculer la probabilité a priori pi_k
pi_AB = num_samples_AB / len(VertebralClas_train)
pi_NO = num_samples_NO / len(VertebralClas_train)

x_test = VertebralVar_test
# Calculer le score des données de test score_NB_test

scores_AB = pi_AB * np.prod([multivariate_normal.pdf(x_test[i], mean=mean_AB[i], cov=Cov_NB_AB[i, i]) for i in range(6)])
scores_NO = pi_NO * np.prod([multivariate_normal.pdf(x_test[i], mean=mean_NO[i], cov=Cov_NB_NO[i, i]) for i in range(6)])

print("scores_AB:", scores_AB)
print("scores_NO:", scores_NO)
# Sélectionnez la catégorie avec le score le plus élevé comme résultat de la prédiction
score_NB_test = np.where(scores_AB > scores_NO, scores_AB, scores_NO)



scores_AB: 5.757446295461836e-127
scores_NO: 0.0


La matrice de confusion est une matrice qui synthétise les performances d'une régle de classification. Chaque ligne correspond à une classe réelle, chaque colonne correspond à une classe estimée. La cellule (ligne L, colonne C) contient le nombre d'éléments de la classe réelle L qui ont été estimés comme appartenant à la classe C. Voir par exemple [ici](https://fr.wikipedia.org/wiki/Matrice_de_confusion).

> Evaluer les performances de la méthode sur l'échantillon test. Vous pourrez utiliser la fonction [`confusion_matrix`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix) de la librairie [`sklearn.metrics`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).

In [10]:
from sklearn.metrics import confusion_matrix

# Calculer la matrice de confusion
cnf_matrix_NB_test = confusion_matrix(VertebralClas_test, pred_NB_test)

# Calculer la matrice de confusion normalisée (exprimée en pourcentage)
cnf_matrix_NB_test_normalized = cnf_matrix_NB_test.astype('float') / cnf_matrix_NB_test.sum(axis=1).reshape(-1,1) 

# Imprimer la matrice de confusion et la matrice de confusion normalisée
print("Confusion Matrix (Unnormalized):\n", cnf_matrix_NB_test)
print("\nConfusion Matrix (Normalized):\n", cnf_matrix_NB_test_normalized)


# cnf_matrix_NB_test.astype('float') / cnf_matrix_test.sum(axis=1).reshape(-1,1) 

Confusion Matrix (Unnormalized):
 [[59 17]
 [ 9 31]]

Confusion Matrix (Normalized):
 [[0.77631579 0.22368421]
 [0.225      0.775     ]]


>  Il existe bien sûr une fonction scikit-learn  pour la méthode Naive Bayes : voir [ici](http://scikit-learn.org/stable/modules/naive_bayes.html). Vérifier que votre prédicteur donne la même réponse de cette fonction.

In [11]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

# Create Gaussian Naive Bayes
gnb = GaussianNB()

# Use training data to fit the model
gnb.fit(VertebralVar_train, VertebralClas_train)

# Use the model to make class predictions on test data
pred_NB_test = gnb.predict(VertebralVar_test)

# Calculate confusion matrix
cnf_matrix_NB_test = confusion_matrix(VertebralClas_test, pred_NB_test)

# Print confusion matrix
print("Confusion Matrix:\n", cnf_matrix_NB_test)



Confusion Matrix:
 [[59 17]
 [ 9 31]]


# Classifieur par plus proches voisins

Il est préférable d'utiliser la structure de données de type [k-d tree](https://en.wikipedia.org/wiki/K-d_tree) pour effectuer des requêtes de plus proches voisins dans un nuage de points. 

> Contruction du k-d tree pour les données train (pour la métrique euclidienne) :

In [12]:
from sklearn.neighbors import KDTree
tree =  KDTree(VertebralVar_train, leaf_size=30)

> Rechercher les 10 plus proches voisins dans les données d'apprentissage du premier point des données de test et afficher les classes de ces observations voisines.

In [13]:
indices_voisins =  tree.query(VertebralVar_test[0].reshape(1, -1), k=10)
print(indices_voisins)
classes_voisins = VertebralClas_train[indices_voisins[1][0]]
print(classes_voisins)    

(array([[ 8.08523964,  9.61650664, 11.4683957 , 12.09108763, 13.6653723 ,
        14.83967318, 15.50358346, 18.32258988, 18.88993912, 19.09660703]]), array([[ 26,   2,  77, 116, 118,  70, 126, 137, 110, 159]], dtype=int64))
['AB' 'AB' 'AB' 'AB' 'AB' 'NO' 'AB' 'AB' 'AB' 'AB']


Pour le classifieur par plus proches vosins, la prediction est la classe majoritaire des k plus proches voisins.

> Donner la prédiction pour le premier point de test par vote majoritaire sur ses 10 plus proches voisins 

In [14]:
from collections import Counter

# Obtenez les indices des 10 voisins les plus proches du premier point de données dans les données de test
indices_voisins = tree.query(VertebralVar_test[0].reshape(1, -1), k=10)[1][0]

# Obtenez les catégories de ces voisins
classes_voisins = VertebralClas_train[indices_voisins]

# Compter les occurrences de chaque catégorie
count_classes = Counter(classes_voisins)

# Trouvez les catégories les plus fréquentes
prediction = count_classes.most_common(1)[0][0]

# Imprimer les résultats de prédiction
print("Prédiction pour le premier point de test :", prediction)


Prédiction pour le premier point de test : AB


> Donner la prediction du classifieur ppv pour toutes les données de test. Evaluer la qualité du classifieur.

In [15]:
from sklearn.metrics import confusion_matrix

# Choisissez une valeur k, qui est le nombre de voisins les plus proches
k_class = 10 

# Initialiser le tableau des résultats de prédiction
pred_kNN_test = []

# Faire des prédictions pour chaque point de données dans les données de test
for i in range(len(VertebralVar_test)):
    # Obtenez les indices des k voisins les plus proches du i-ème point de données dans les données de test
    indices_voisins = tree.query(VertebralVar_test[i].reshape(1, -1), k=k_class)[1][0]
    
    # Obtenez les catégories de ces voisins
    classes_voisins = VertebralClas_train[indices_voisins]
    
    # Compter les occurrences de chaque catégorie
    count_classes = Counter(classes_voisins)
    
    # Trouvez la catégorie avec le plus d'occurrences comme résultat de la prédiction
    prediction = count_classes.most_common(1)[0][0]
    
    # Ajouter les résultats de prédiction au tableau de prédiction
    pred_kNN_test.append(prediction)

# Calculer la matrice de confusion
cnf_matrix_kNN = confusion_matrix(VertebralClas_test, pred_kNN_test)

# Calculer la matrice de confusion normalisée
cnf_matrix_kNN_normalized = cnf_matrix_kNN.astype('float') / cnf_matrix_kNN.sum(axis=1)[:, np.newaxis]

# Imprimer la matrice de confusion
print("Confusion Matrix:\n", cnf_matrix_kNN)
print("\nNormalized Confusion Matrix:\n", cnf_matrix_kNN_normalized)



# k_class = ### CHOISIR  ####  #nombre de plus proche voisins utilisés
# pred_kNN_test =  ### TO DO ####
# cnf_matrix_kNN =### TO DO ####
# cnf_matrix_kNN.astype('float') / cnf_matrix_kNN.sum(axis=1).reshape(-1,1) 

Confusion Matrix:
 [[66 10]
 [ 7 33]]

Normalized Confusion Matrix:
 [[0.86842105 0.13157895]
 [0.175      0.825     ]]


Il existe bien sûr une fonction scikit-learn pour le classifieur plus proche voisin, voir [ici](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).